<a href="https://colab.research.google.com/github/danielbauer1979/CAS_PredMod/blob/main/pa_pynb_sess9_AlgFairness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 9: Algorithmic Bias and Fairness

Jim Guszcza and Dani Bauer, 10/2023

In this tutorial, we discuss approaches how to analyze a predictive algorithm with regards to "fairness." We do so in the context of a well-known case study on an algorithm that assists judges in parole decisions. We go over different notions of fairness, discuss tradeoffs, and explain the intuition behind the results of the analyses. We also discuss approaches how to adjust algorithms to enforce fairness. A second case study, which we will ask you to work on, illustrates how these ideas apply in the actuarial context.

### Load required packages

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_curve, auc

In [ ]:
!pip install aequitas
#Another library that seems to be popular is fariness 360:
#!pip install aif360

In [4]:
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
import aequitas.plot as ap

# Compas Case Study

The Compas data originates from a [well-known case study on algorithmic bias](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing). The background is [as follows](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm): Across the nation, judges, probation and parole officers are increasingly using algorithms to assess a criminal defendant's likelihood of becoming a recidivist---a term used to describe criminals who re-offend. One of the commercial tools made by Northpointe, Inc. is called COMPAS (which stands for Correctional Offender Management Profiling for Alternative Sanctions). The case study compares outcomes and risk scores for individuals belonging to different races. In what follows, we will go through some of these analyses ourselves.

## The Compas Data

The data is in our github folder. Let's take a look:

In [ ]:
!git clone https://github.com/danielbauer1979/CAS_PredMod.git

In [ ]:
dat = pd.read_csv('CAS_PredMod/pa_data_compasdata.csv')
dat.head()

In [ ]:
dat.describe()

The data contains information on recidivism of 6,172 individuals as well as information on the individual's age, sex, criminal history, their ethnicity---and the risk score they received from the COMPAS algorithm.

To simplify the situation, we focus on two ethnicity levels only: African-Americans and Caucasians.

In [ ]:
dat = dat.loc[(dat['ethnicity'] == 'Caucasian') | (dat['ethnicity'] == 'African_American')]
dat.describe()

So we still have the majority of individuals included. We will consider the African-Americans as the "protected" class.

We commence by exploring the data some and looking at fairness manually, but then we will also explore how to use the Aequitas package in this setting.

### Fairness scores vs. recidivism rates

Let's start by comparing the COMPAS scores between the two ethic groups (see also the density plots from the fairness package above):

In [ ]:
dat.loc[dat['ethnicity'] == 'Caucasian']['probability']

In [ ]:
plt.title("Score distribution by group")
plt.xlabel("Group: 1 = Caucasian, 2 = African-American")
plt.boxplot([dat.loc[dat['ethnicity'] == 'Caucasian']['probability'],dat.loc[dat['ethnicity'] == 'African_American']['probability']])
plt.show()

The distribution of scores in the African-American group has a higher median and higher percentiles than the scores in the Caucasian group.

However, consider the number of re-offenders between the group, we see the following:

In [ ]:
aq_palette = sns.diverging_palette(225, 35, n=2)
by_race = sns.countplot(x="ethnicity", hue="Two_yr_Recidivism", data=dat[dat.ethnicity.isin(['African_American', 'Caucasian'])], palette=aq_palette)

Let's consider raw averages of re-offenders in the two groups -- for caucasians:

In [ ]:
np.average(dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == 'yes')

And for the protected group:

In [ ]:
np.average(dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == 'yes')

So, we can interpret the higher average COMPAS scores as resulting from statistical facts in this particular population.

Let’s investigate accuracy, i.e., how accurate the algorithm is, on aggregate and by protected vs. non-protected class. In doing so, we determine the correctly classified re-offenders (true positives), the correctly cassified non-re-offenders (true negatives), the incorrectly classified re-offenders (false negatives), and the incorrectly classified non-re-offenders (false positives)—by class.

In [34]:
TP_0 = sum((dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == "yes") * dat.loc[dat['ethnicity'] == 'Caucasian']['predicted'])
TP_1 = sum((dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == "yes") * dat.loc[dat['ethnicity'] == 'African_American']['predicted'])
FP_0 = sum((dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == "no") * dat.loc[dat['ethnicity'] == 'Caucasian']['predicted'])
FP_1 = sum((dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == "no") * dat.loc[dat['ethnicity'] == 'African_American']['predicted'])
FN_0 = sum((dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == "yes") * (dat.loc[dat['ethnicity'] == 'Caucasian']['predicted'] == 0))
FN_1 = sum((dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == "yes") * (dat.loc[dat['ethnicity'] == 'African_American']['predicted'] == 0))
TN_0 = sum((dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == "no") * (dat.loc[dat['ethnicity'] == 'Caucasian']['predicted'] == 0))
TN_1 = sum((dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == "no") * (dat.loc[dat['ethnicity'] == 'African_American']['predicted'] == 0))

"Accuracy" now is simply the correctly classified individuals divided by all individuals.

In [ ]:
acc = (TP_0+TP_1+TN_0+TN_1)/(TP_0 + FP_0 + TP_1 + FP_1 + TN_0 + FN_0 + TN_1 + FN_1)
acc

and by group:

In [ ]:
acc_0 = (TP_0+TN_0)/(TP_0 + FP_0 + TN_0 + FN_0)
acc_0

and for the protected group:

In [ ]:
acc_1 = (TP_1+TN_1)/(TP_1 + FP_1 + TN_1 + FN_1)
acc_1

In [ ]:
plt.bar(['Caucasian','African-American'], [acc_0,acc_1], color ='maroon', width = 0.4)
plt.xlabel("Class")
plt.ylabel("Accuracy")
plt.title("Accuracy:    (TP+TN) / (TP+FP+TN+FN)")
plt.show()

It appears that the algorithm is similarly "accurate" for both groups, and in fact the accuracy is even somewhat higher for the protected group.

Hence, one may conclude that the algorithm performs reasonably well---and that there are no major concerns with regards to differential performance for the protected group. In fact, this may have been the mindset by the creators of the COMPAS algorithm.

### Alternative Fairness Metrics

While "accuracy"---or one minus accuracy, which is the misclassification rate---do not point to problems, we can investigate more differentiated error metrics. One obvious way of doing this is by comparing

We can further assess the performance by visualizing false negative rates and the true positive rates by group. One way of looking at this is how many of the negatively classified individuals (low risk) were labeled so incorrectly, and how many of the positively classified (high risk) were labeled so correctly. The former proportion of negatively classified for which outcome indeed was one is also referred to as the *False Omission Rate*, whereas the latter proportion of positively classified that were indeed correct is also referred to as *Precision*:

In [ ]:
aq_palette = sns.diverging_palette(225, 35, n=2)
data_tmp = [['False Omission Rate',FN_0/(TN_0+FN_0),'Caucasian'],['False Omission Rate',FN_1/(TN_1+FN_1),'African_American'],['Precision',TP_0/(TP_0+FP_0),'Caucasian'],['Precision',TP_1/(TP_1+FP_1),'African_American']]
df_tmp = pd.DataFrame(data_tmp, columns=['Metric','rate','ethnicity'])
sns.barplot(df_tmp, x="Metric", y="rate", hue="ethnicity", palette=aq_palette)

We start seeing some differences between the groups, although the differences are not startling. The algorithm seems to me more “precise” for the protected class.

Differences between the groups start to emerge if we consider indviduals that are labeled as being likely to re-offend, i.e., the total fraction of positives between the two groups: